In [1]:
from pandas import read_csv
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import copy
import itertools
import time
import csv
import random
import json

%matplotlib inline

In [2]:
training = read_csv("training_2140.csv")

Изображение записано в виде строки чисел от 0 до 255, разделенных пробелом. Преобразуем эту строку к массиву чисел 0-255

In [3]:
training.Image = training.Image.apply(lambda img: np.fromstring(img, sep=' '))

In [4]:
training = training.drop(training.columns.values[0], 1)

labels - правильные координаты признаков, используемые для обучения сети

In [5]:
t_labels = np.array([])
for key_name in training.columns.values[:-1]:
    t_labels = np.append(t_labels, np.array(training[key_name]))
t_labels = t_labels.reshape(training.shape[1]-1, len(training.Image)).T
t_labels = t_labels[:2048]

Для примера выведем на экран размерность массива признаков в виде (кол-во строк; кол-во признаков для одного изображения).
И также выведем признаки для первого изображения

In [6]:
print t_labels.shape
print t_labels[0]

(2048, 30)
[ 66.03356391  39.00227368  30.22700752  36.4216782   59.58207519
  39.64742256  73.13034586  39.96999699  36.35657143  37.3894015
  23.45287218  37.3894015   56.95326316  29.03364812  80.22712782
  32.22813835  40.22760902  29.0023218   16.35637895  29.64747068
  44.42057143  57.06680301  61.19530827  79.97016541  28.61449624
  77.38899248  43.3126015   72.93545865  43.13070677  84.48577444]


In [7]:
swap_indices = [(0,2), (1,3), (4,8), (5,9), (6,10), (7,11), (12,16), (13,17), (14,18), (15,19), (22,24), (23,25)]
def swap(l,i,j):
    t = np.copy(l[:,i])
    l[:,i] = l[:,j]
    l[:,j] = t

Функция для вывода изображения на экран

In [8]:
def print_image(data):
    img = [[x]*3 for x in data]
    img = np.reshape(img, (96,96,3))
    plt.imshow(img)

Функция для печати изображения с нанесенными признаками

In [9]:
def test_point(img_ind, a, b, length):
    plt.xlim([0, 96])
    plt.ylim([96, 0])
    f = teX[img_ind].reshape(96*96)
    print_image(f)
    for i in range(length):
        plt.plot(a[img_ind][i][0],a[img_ind][i][1], 'r*')
        plt.plot(b[img_ind][i][0],b[img_ind][i][1], 'bo')
    plt.show()

Функции для масштабирование признаков в прямую ((1; 96) -> (-1; 1)) и обратную сторону

In [10]:
def scale(x):
    return (x - 48) / 48
def unscale(x):
    return x * 48 + 48

Объявление массива образцов для обучения и тестирования, печать его размерности и первого элемента

In [11]:
t_samples = np.array([np.copy(img) for img in training.Image[:2048].apply(lambda x: x / 255.0)])
print t_samples.shape
print t_samples[0]

(2048, 9216)
[ 0.93333333  0.9254902   0.92941176 ...,  0.2745098   0.29411765
  0.35294118]


In [12]:
def swap_samples(samples):
    res = np.copy(samples)
    for img in res:
        img = img.reshape(96,96)
        for i in range(48):
            swap(img,i,95-i)
        img = img.reshape(96*96,1)
    return res
swaped_samples = swap_samples(t_samples)

In [13]:
def swap_labels(labels, swap_indices):
    res = np.copy(labels)
    for i,j in swap_indices:
        swap(res,i,j)
    res[:,0::2] = 96 - res[:,0::2]
    return res
swaped_labels = swap_labels(t_labels,swap_indices)
print swaped_labels.shape

(2048, 30)


In [14]:
samples = np.vstack((t_samples, swaped_samples))
labels = np.vstack((t_labels, swaped_labels))
print samples.shape, '\n', labels.shape

(4096, 9216) 
(4096, 30)


Масштабирование признаков и печать первого элемента до масштабирования и после

In [15]:
print labels[0]
labels = scale(labels)
print labels[0]

[ 66.03356391  39.00227368  30.22700752  36.4216782   59.58207519
  39.64742256  73.13034586  39.96999699  36.35657143  37.3894015
  23.45287218  37.3894015   56.95326316  29.03364812  80.22712782
  32.22813835  40.22760902  29.0023218   16.35637895  29.64747068
  44.42057143  57.06680301  61.19530827  79.97016541  28.61449624
  77.38899248  43.3126015   72.93545865  43.13070677  84.48577444]
[ 0.37569925 -0.18745263 -0.37027068 -0.24121504  0.24129323 -0.17401203
  0.52354887 -0.16729173 -0.24257143 -0.22105414 -0.5113985  -0.22105414
  0.18652632 -0.39513233  0.6713985  -0.32858045 -0.16192481 -0.39578496
 -0.65924211 -0.38234436 -0.07457143  0.18889173  0.27490226  0.66604511
 -0.40386466  0.61227068 -0.09765414  0.51948872 -0.10144361  0.7601203 ]


Задание размеров обучающей и тестирующей выборки и вывод их на экран

In [16]:
full_volume = len(samples)
train_volume = int(full_volume * 0.875)
test_volume = full_volume - train_volume
print "%d\n%d\n%d" % (full_volume, train_volume, test_volume)

4096
3584
512


Задание обучающей и тестирующей выборки

In [17]:
trX = np.array(samples[:train_volume], dtype=np.float)
teX = np.array(samples[train_volume:full_volume], dtype=np.float)
trY = np.array(labels[:train_volume], dtype=np.float)
teY = np.array(labels[train_volume:full_volume], dtype=np.float)

trX = trX.reshape(-1, 96, 96, 1)  # 96x96x1 input img
teX = teX.reshape(-1, 96, 96, 1)  # 96x96x1 input img

print trX.shape
print trY.shape
print teX.shape
print teY.shape

(3584, 96, 96, 1)
(3584, 30)
(512, 96, 96, 1)
(512, 30)


In [18]:
def read_populations(populations):
    configs = []
    for p in populations:
        if p['optimizer'] == "RMSProp":
            opt = tf.train.RMSPropOptimizer
        elif p['optimizer'] == "Adam":
            opt = tf.train.AdamOptimizer
        else:
            opt = tf.train.AdamOptimizer
        configs.append([opt, p['convs'], p['fc']])
    return configs

In [19]:
def read_configs(fname):
    epochs, eps, batch_size = 10, 4, 128
    keep_prob_conv, keep_prob_fc = 0.8, 0.5
    configs = [[1, [3,3,3,3],[512]],
               [0, [4,3,2,1],[700]]]
    with open(fname) as f:
        cf = json.load(f)
    if cf.has_key('epochs'):
        epochs_count = cf['epochs']
    if cf.has_key('eps'):
        eps = cf['eps']
    if cf.has_key('batch_size'):
        batch_size = cf['batch_size']
    if cf.has_key('keep_prob_conv'):
        keep_prob_conv = cf['keep_prob_conv']
    if cf.has_key('keep_prob_fc'):
        keep_prob_fc = cf['keep_prob_fc']
    if cf.has_key('population'):
        configs = read_populations(cf['population'])
    return (epochs, eps, batch_size, keep_prob_conv, keep_prob_fc, configs)

In [20]:
epo,eps,bs,kpc,kpf,cfgs = read_configs("config.json")

Задание eps для проверки точности работы нейросети, а также размеров одновременно подаваемых образцов для обучения (они подаются нейросети, вычисляется средняя ошибка для всех этих образцов и только после этого меняются веса) и тестирования

In [21]:
eps = 4 # treshold = 4 pixels
batch_size = 128

Функции для оценки точности. Проверяется что все сравниваемые элементы отличаются не более чем на eps

In [22]:
def eq_eps(y, yl):
    uy = unscale(y)
    uyl = unscale(yl)
    return np.abs(uy - uyl) < eps

def all_eq_eps(y, yl):
    return (all(eq_eps(yc, ylc)) for yc, ylc in zip(y, yl))

In [23]:
class ConfigNN:
    def __init__(self, input_img_size, output_count, opt, cost, lrn_rate,
                 changing_learning_rate, conv_kernels, pooling_kernels,
                 fc_layers, keep_prob_conv, keep_prob_fc,
                 batch_size, train_volume, use_bias, epochs):
        # img_size = 96x96, not (-1, 96, 96, 1)
        # pooling_kernels = [...,(1,2,2,1), ...]
        # conv_kernel = [width, height, count]
        self.input_img_size = input_img_size # ?
        self.output_count = output_count
        self.optimizer = opt
        self.train_volume = train_volume
        self.batch_size = batch_size
        self.epochs = epochs
        self.changing_learning_rate = changing_learning_rate
        self.use_bias = use_bias
        self.best_cost = 1000000
        self.best_epoch = 0
        
        self.pooling_kernels_shapes =copy.deepcopy(pooling_kernels)
        
        self.keep_prob_conv = keep_prob_conv
        self.keep_prob_fc = keep_prob_fc
        
        self._create_weights(conv_kernels, fc_layers)
        
        self.X = tf.placeholder("float",[None, self.input_img_size[0], self.input_img_size[1], 1], name="X")
        self.Y = tf.placeholder("float",[None, self.output_count], name="Y")
        
        if changing_learning_rate:
            self.global_step = tf.Variable(1, trainable=False)
            self.learning_rate = tf.train.exponential_decay(lrn_rate, self.global_step * batch_size,
                                           train_volume, 0.96, staircase=True)
        else:
            self.learning_rate = lrn_rate
            
        self.predict_op = self._predict(self.X)
        self.cost = cost(self.Y - self.predict_op)
        if changing_learning_rate:
            self.train_op = self.optimizer(self.learning_rate).minimize(self.cost, global_step=self.global_step)
        else:
            self.train_op = self.optimizer(self.learning_rate).minimize(self.cost)
            
    def print_info(self):
        #print 'image size: ', self.input_img_size
        #print 'outputs count: ', self.output_count
        print 'optimizer: ', self.optimizer
        #print 'train volume: ', self.train_volume
        #print 'batch size: ', self.batch_size
        print 'convolutional kernel shapes: ', self.conv_kernels_shapes
        print 'pooling kernels shapes: ', self.pooling_kernels_shapes
        print 'fully connected shapes: ', self.fc_layers_shapes
        #print 'keep_prob for conv layers: ', self.keep_prob_conv
        #print 'keep_prob for fc layers: ', self.keep_prob_fc
        #print 'use bias: ', self.use_bias
    
    def _init_weights(self, shape):
        return tf.Variable(tf.truncated_normal(shape, stddev=0.01))
    
    def _create_weights(self, conv_kernels, fc_layers):
        # предполагается что ядра свертки и субдискретизации имеют размеры NxN и их кол-во совпадает        
        self.conv_kernels_shapes = copy.deepcopy(conv_kernels)
        self.fc_layers_shapes = copy.deepcopy(fc_layers)
        
        self.conv_kernels_shapes[0][2:] = [1, self.conv_kernels_shapes[0][2]]
        for i in range(1, len(self.conv_kernels_shapes)):
            self.conv_kernels_shapes[i][2:] = [self.conv_kernels_shapes[i-1][-1], self.conv_kernels_shapes[i][2]]
            
        img_sz = self.input_img_size[0]
        for pk in self.pooling_kernels_shapes:
            img_sz /= pk[1]
            
        conv_output_count = self.conv_kernels_shapes[-1][3] * img_sz * img_sz
        self.fc_layers_shapes[0] = [conv_output_count, self.fc_layers_shapes[0]]

        for i in range(1, len(self.fc_layers_shapes)):
            self.fc_layers_shapes[i] = [self.fc_layers_shapes[i-1][1], self.fc_layers_shapes[i]]

        self.conv_kernels = [self._init_weights(sh) for sh in self.conv_kernels_shapes]
        self.fc_layers = [self._init_weights(sh) for sh in self.fc_layers_shapes]
        
        if self.use_bias:
            self.conv_biases = [tf.Variable(tf.zeros([sh[-1]])) for sh in self.conv_kernels_shapes]
            self.fc_biases = [tf.Variable(tf.constant(0.1, shape=[sh[-1]])) for sh in self.fc_layers_shapes]
    
    def _predict(self, X):
        y = X
        for i in range(len(self.conv_kernels)):
            y = tf.nn.conv2d(y, self.conv_kernels[i], strides=[1,1,1,1], padding="SAME")
            if self.use_bias:
                y = tf.nn.bias_add(y, self.conv_biases[i])
            y = tf.nn.relu(y)
            pooling = self.pooling_kernels_shapes[i]
            y = tf.nn.max_pool(y, pooling, pooling, padding="SAME")
            y = tf.nn.dropout(y, self.keep_prob_conv)
        y = tf.reshape(y, [-1, self.fc_layers_shapes[0][0]])
        for i in range(len(self.fc_layers)-1):
            l = self.fc_layers[i]
            y = tf.matmul(y, l)
            if self.use_bias:
                y += self.fc_biases[i]
            y = tf.nn.relu(y)
            y = tf.nn.dropout(y, self.keep_prob_fc)
        y = tf.matmul(y, self.fc_layers[-1])
        return y

In [24]:
sess = tf.Session()

In [25]:
def run(cn):
    try:
        sess.run(tf.initialize_all_variables())
        training_batch = zip(range(0, cn.train_volume, cn.batch_size),
                         range(cn.batch_size, cn.train_volume, cn.batch_size))

        kp_conv = cn.keep_prob_conv
        kp_fc = cn.keep_prob_fc
        if not cn.changing_learning_rate:
            cur_learning_rate = cn.learning_rate
        shuffled_index = range(len(trX))
            
        for i in range(cn.epochs):
            random.shuffle(shuffled_index)
            trX_shuffled = [trX[j] for j in shuffled_index]
            trY_shuffled = [trY[j] for j in shuffled_index]
            
            for start, end in training_batch:
        
                feed_dict={cn.X: trX_shuffled[start:end], cn.Y: trY_shuffled[start:end]}
                if cn.changing_learning_rate:
                    _, cst, cur_learning_rate = sess.run([cn.train_op, cn.cost, cn.learning_rate], feed_dict=feed_dict)
                else:
                    _, cst = sess.run([cn.train_op, cn.cost], feed_dict=feed_dict)
                
            cn.keep_prob_conv = 1.0
            cn.keep_prob_fc = 1.0
            y = sess.run(cn.predict_op, feed_dict={cn.X: teX[:256]})
            cn.keep_prob_conv = kp_conv
            cn.keep_prob_fc = kp_fc

            acc = [eq_eps(a,b) for (a,b) in zip(teY[:256], y)]
            accuracy = np.mean(acc)
            print "Epoch: %d;   Cost: %7.3f;   Accuracy: %.0f%%;   Learning rate: %.5f" % (i, cst, accuracy*100, cur_learning_rate)
            
            if cst < cn.best_cost:
                cn.best_cost = cst
                cn.best_epoch = i
                cn.accuracy = accuracy
            if i - cn.best_epoch > 7:
                print "Early stop"
                print "Best cost: %7.3f;   Best epoch: %d" % (cn.best_cost, cn.best_epoch)
                break
        
        cn.keep_prob_conv = 1.0
        cn.keep_prob_fc = 1.0
        y = sess.run(cn.predict_op, feed_dict={cn.X: teX[:5]})
        cn.keep_prob_conv = kp_conv
        cn.keep_prob_fc = kp_fc
        a, b = unscale(y[0:5]), unscale(teY[0:5])
        length = len(a[0]) / 2
        a = a.reshape(5, length, 2)
        b = b.reshape(5, length, 2)
        for i in range(5):
            test_point(i, a, b, length)
    except Exception as e:
        print "===============\nERROR: %s\n===============" % e

In [26]:
def rand_bool():
    return bool(random.getrandbits(1))

def mutate(conf):
    config = copy.deepcopy(conf)
    
    m_conv, m_fc = random.random(), random.random()
    config[0] = random.randint(0,1) # 0 or 1 - Adam or RMSProp
    
   
    if m_conv < 0.5:
        change = random.randint(0, 3)
        i_conv = random.randint(0, len(config[1])-1)
        config[1][i_conv] = random.randint(1,5)
    
    if m_fc < 0.5:
        change = random.randint(0, 3)
        i_fc = random.randint(0, len(config[2])-1)
        if change == 0 and len(config[2]) > 1: # delete
            del config[2][i_fc]
        elif change == 1 and len(config[2]) < 5: # add
            config[2][i_fc:i_fc] = [random.randint(200,1200)]
        elif change == 2: #change value
            config[2][i_fc] = random.randint(200,1200)
            
    return config

In [27]:
def convolution_fullshape(size, count):
    return [size, size, count]

def conv_layers_shapes(sizes):
    conv_count = 16
    convs = []
    for conv_sz in sizes:
        shape = convolution_fullshape(conv_sz, conv_count)
        convs.append(shape)
        conv_count *= 2
    return convs

optimizers = [tf.train.RMSPropOptimizer, tf.train.AdamOptimizer]

mse_cost = lambda x: tf.reduce_mean(tf.reduce_sum(tf.square(x)))

pooling2 = [1,2,2,1]
pooling3 = [1,3,3,1]
pooling_kernels = [pooling3, pooling2, pooling2, pooling2]

In [28]:
#format of configuration: config[i] = [optimizer_index, [c1, c2, c3], [l1,...lk]]
configs = [[0, [3, 3, 3, 3], [512, 512]],
           [1, [3, 3, 3, 3], [512, 512]],
           [0, [3, 3, 3, 3], [700]],
           [1, [3, 3, 3, 3], [700]]]

In [29]:
def run_on_configs(configs):
    results = []
    k = 0
    for config in configs:
        k += 1
        res = [None, None, None, None, config[0], config[1], config[2]]
        conv_kernels = conv_layers_shapes(config[1])
        opt_ind = config[0]
        cn = ConfigNN([96,96], 30, optimizers[opt_ind], mse_cost, 0.001, True,
                      conv_kernels=conv_kernels,
                      pooling_kernels=pooling_kernels,
                      fc_layers=config[2]+[30],
                      keep_prob_conv=0.8,
                      keep_prob_fc=0.5,
                      batch_size=batch_size, train_volume=train_volume,
                      use_bias=True, epochs=10)
        print '    k = ', k
        cn.print_info()
        start_time = time.clock()
        run(cn)
        t = time.clock() - start_time
        res[3] = t
        #print 'Time(s): ', t

        try:
            res[0] = cn.best_epoch
            res[1] = cn.best_cost
            res[2] = cn.accuracy
        except Exception as e:
            print e

        results.append(res)
    return results
    

In [30]:
def print_configs(configs):
    for config in configs:
        print config
        
def print_in_file(num, data):
    fname = 'solve_genet_'+str(num)+'.csv'
    with open(fname, 'w') as fp:
        cw = csv.writer(fp, delimiter=';')
        cw.writerows(data)

In [32]:
num = 1

half = len(configs)
m_configs = [mutate(c) for c in configs]
configs = configs + m_configs

for i in range(5):
    num += 1
    print_configs(configs[half:])
    results[half:] = run_on_configs(configs[half:])
    
    results.sort(key=lambda x: x[1])
    
    print_in_file(num, results)
    
    configs = [res[half:] for res in results]
    
    m_configs = [mutate(c) for c in configs[:half]]
    configs[half:] = m_configs

Закрытие сессии tensorflow

In [33]:
sess.close()